# Imports:

In [1]:
import pickle
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy.special import expit, logit
from tqdm import trange

import warnings
# Ignore all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append("../Utils/")
from data_generating_utils import generate_population_data
from ddc_utils import (
    compute_all_three_logistic_models,
    compute_average_jn,
    get_pop_gs_for_binary_y,
)

# Hyperparams:

In [2]:
population_size = 100_000
sample_size = 1_000
num_iters_per_population = 10_000

true_beta = 1
link = 'Probit'

In [3]:
# for biased sampling scheme:
sample_probability_centering = 0.8
sample_probability_bias_factor = 1

In [4]:
# for seeding + saving:
pop_index = 1
iter_val = 0
rand_generator = np.random.default_rng(seed=333 * pop_index + iter_val)

# Run:

In [5]:
# hypervariabes where things will be saved as key of sample size -> list.
all_jns_per_sample_size_biased = defaultdict(list)
all_ddc_per_sample_size_biased = defaultdict(list)
all_sample_beta_per_sample_size_biased = defaultdict(list)

all_jns_per_sample_size_full = defaultdict(list)
all_ddc_per_sample_size_full = defaultdict(list)
all_sample_beta_per_sample_size_full = defaultdict(list)

all_realized_sample_sizes_per_sample_size = defaultdict(list)
all_pop_beta_per_sample_size = defaultdict(list)

In [6]:
pop_data = generate_population_data(
    population_size,
    rand_generator,
    true_beta=true_beta,
    link=link,
)

pop_data["marginal_probs"] = expit(
    logit(sample_probability_centering)
    + (2 * pop_data["y"] - 1) * pop_data["x_0"]
)

In [7]:
pickle_filename = f'base_population_data_{link}_{true_beta}.pickle'
with open(pickle_filename, 'wb') as handle:
    pickle.dump(pop_data, handle)

In [8]:
model_iteration = [(0, 'Logit'), (1, 'Probit'), (2, 'CLogLog')]

## Get population-level statistics:

In [9]:
pop_x = pop_data[['x_0']]
pop_y = pop_data['y']

# compute the population models, saving their betas and gs:
population_models = (
    compute_all_three_logistic_models(pop_x, pop_y)
)

pop_logit_model, pop_probit_model, pop_cloglog_model = population_models

pop_betas = [
    np.array(pop_logit_model.params),
    np.array(pop_probit_model.params),
    np.array(pop_cloglog_model.params),
]

pop_gs = get_pop_gs_for_binary_y(population_models, pop_x, pop_y)

## Iterate per sample size:

In [10]:
for _ in trange(num_iters_per_population, mininterval=10):
    # sample the ideal and biased sample:
    pop_data["r0"] = 0
    pop_data["r"] = 0
    pop_data.loc[
        np.random.choice(pop_data.index, size=sample_size, replace=False),
        "r0",
    ] = 1
    full_sampled_data = pop_data[pop_data["r0"] == 1]
    
    other_sample_indices = full_sampled_data.index[
        rand_generator.binomial(n=1, p=full_sampled_data["marginal_probs"]) == 1
    ]
    pop_data.loc[other_sample_indices, "r"] = 1
    
    # sample_data here means the biased sample data.
    sample_data = pop_data[pop_data["r"] == 1]
    realised_sample_size = len(other_sample_indices)

    # compute biased x, y, model, beta
    sample_x = sample_data[['x_0']]
    sample_y = sample_data["y"]
    sample_models = compute_all_three_logistic_models(sample_x, sample_y)
    sample_betas = [sample_model.params for sample_model in sample_models]
    sample_r = pop_data["r"]

    # compute full x, y, model, beta
    sample_x_full = full_sampled_data[['x_0']]
    sample_y_full = full_sampled_data["y"]
    sample_models_full = compute_all_three_logistic_models(
        sample_x_full, sample_y_full
    )
    sample_betas_full = [
        sample_model_full.params for sample_model_full in sample_models_full
    ]
    sample_r_full = pop_data["r0"]

    # compute biased versions of things:
    all_sample_beta_per_sample_size_biased[sample_size].append(
        [pd.Series(sample_beta) for sample_beta in sample_betas]
    )
    all_ddc_per_sample_size_biased[sample_size].append(
        [
            pop_gs[link_fn].corrwith(sample_r)[["x_0"]]
            for _, link_fn in model_iteration
        ]
    )
    all_jns_per_sample_size_biased[sample_size].append(
        [
            compute_average_jn(
                pop_betas[model_index],
                sample_betas[model_index],
                sample_x,
                sample_y,
                link_fn=link_fn,
            )
            for model_index, link_fn in model_iteration
        ]
    )

    all_realized_sample_sizes_per_sample_size[sample_size].append(
        realised_sample_size
    )

    # compute full versions of things:
    all_sample_beta_per_sample_size_full[sample_size].append(
        [pd.Series(sample_beta) for sample_beta in sample_betas_full]
    )
    all_ddc_per_sample_size_full[sample_size].append(
        [
            pop_gs[link_fn].corrwith(sample_r_full)[["x_0"]]
            for _, link_fn in model_iteration
        ]
    )
    all_jns_per_sample_size_full[sample_size].append(
        [
            compute_average_jn(
                pop_betas[model_index],
                sample_betas_full[model_index],
                sample_x_full,
                sample_y_full,
                link_fn=link_fn,
            )
            for model_index, link_fn in model_iteration
        ]
    )

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:13<00:00, 31.89it/s]


C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

  3%|██▌                                                                           | 330/10000 [00:32<15:45, 10.23it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  r

C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\array

C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\array

C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\array

...

C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\array

C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 89%|████████████████████████████████████████████████████████████████████▋        | 8921/10000 [15:00<02:00,  8.96it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  r

C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 92%|██████████████████████████████████████████████████████████████████████▉      | 9217/10000 [15:30<01:21,  9.57it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  r

C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 95%|████████████████████████████████████████████████████████████████

C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 98%|███████████████████████████████████████████████████████████████████████████▋ | 9824/10000 [16:31<00:17,  9.82it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  r

## save data:

In [11]:
all_raw_data = [
    all_jns_per_sample_size_biased,
    all_ddc_per_sample_size_biased,
    all_sample_beta_per_sample_size_biased,
    all_jns_per_sample_size_full,
    all_ddc_per_sample_size_full,
    all_sample_beta_per_sample_size_full,
    all_realized_sample_sizes_per_sample_size,
    all_pop_beta_per_sample_size,
]

In [12]:
pickle_filename = f'all_raw_data_{link}_{true_beta}.pickle'
with open(pickle_filename, 'wb') as handle:
    pickle.dump(all_raw_data, handle)

In [13]:
pickle_filename = f'pop_betas_{link}_{true_beta}.pickle'
with open(pickle_filename, 'wb') as handle:
    pickle.dump(pop_betas, handle)